In [ ]:
"""
Akkadian to English Translation Inference
ByT5 Optimized — tuned decoding for better score
"""

import re
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# ──────────────────────────────────────────────────────────────────────────────
# CONFIGURATION
# ──────────────────────────────────────────────────────────────────────────────
INFERENCE_CONFIG = {
    "test_data_path": "/kaggle/input/deep-past-initiative-machine-translation/test.csv",
    "trained_model_path": "/kaggle/input/final-byt5/byt5-akkadian-optimized-34x",
    "computation_device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "maximum_sequence_length": 512,
    "inference_batch_size": 8,

    # 🔹 Tuned decoding (small but effective changes)
    "generation_parameters": {
        "num_beams": 12,
        "max_new_tokens": 600,
        "min_new_tokens": 20,          # NEW: avoids empty / trivial outputs
        "length_penalty": 1.05,        # ↑ better for Akkadian → English expansion
        "early_stopping": True,        # NEW: prevents tail hallucination
        "repetition_penalty": 1.15,    # NEW: better than n-gram blocking for ByT5
    }
}

# ──────────────────────────────────────────────────────────────────────────────
# INPUT PREPROCESSING
# ──────────────────────────────────────────────────────────────────────────────
def preprocess_input_text(text):
    if pd.isna(text):
        return ""
    text = str(text)
    text = re.sub(r'(\.{3,}|…+|……)', '<big_gap>', text)
    text = re.sub(r'(xx+|\s+x\s+)', '<gap>', text)
    return text

# ──────────────────────────────────────────────────────────────────────────────
# OUTPUT POSTPROCESSING
# ──────────────────────────────────────────────────────────────────────────────
def postprocess_translation_output(text):
    if not isinstance(text, str) or not text.strip():
        return ""

    # Normalize special characters
    text = text.replace('ḫ', 'h').replace('Ḫ', 'H')
    text = text.translate(str.maketrans("₀₁₂₃₄₅₆₇₈₉", "0123456789"))

    # Normalize gaps
    text = re.sub(r'(\[x\]|\(x\)|\bx\b)', '<gap>', text, flags=re.I)
    text = re.sub(r'(\.{3,}|…|\[\.+\])', '<big_gap>', text)

    # Merge adjacent gaps
    text = re.sub(r'<gap>\s*<gap>', ' <big_gap> ', text)
    text = re.sub(r'<big_gap>\s*<big_gap>', ' <big_gap> ', text)

    # Remove grammatical annotations
    text = re.sub(
        r'\((fem|plur|pl|sing|singular|plural|\?|!)\.?\s*\w*\)',
        '',
        text,
        flags=re.I
    )

    # Protect gap tokens
    text = text.replace('<gap>', '\x00GAP\x00')
    text = text.replace('<big_gap>', '\x00BIG\x00')

    # Remove unwanted characters
    forbidden = '!?()"—–<>⌈⌋⌊[]+ʾ/;'
    text = text.translate(str.maketrans('', '', forbidden))

    # Restore gaps
    text = text.replace('\x00GAP\x00', ' <gap> ')
    text = text.replace('\x00BIG\x00', ' <big_gap> ')

    # Decimal → fraction normalization
    fraction_map = {
        r'\.5\b': ' ½',
        r'\.25\b': ' ¼',
        r'\.75\b': ' ¾',
        r'\.33+\d*\b': ' ⅓',
        r'\.66+\d*\b': ' ⅔'
    }
    for pat, sym in fraction_map.items():
        text = re.sub(r'(\d+)' + pat, r'\1' + sym, text)
        text = re.sub(r'\b0' + pat, sym.strip(), text)

    # Remove repetitions
    text = re.sub(r'\b(\w+)(?:\s+\1\b)+', r'\1', text)
    for n in range(4, 1, -1):
        pat = r'\b((?:\w+\s+){' + str(n-1) + r'}\w+)(?:\s+\1\b)+'
        text = re.sub(pat, r'\1', text)

    # Cleanup spacing
    text = re.sub(r'\s+([.,:])', r'\1', text)
    text = re.sub(r'([.,])\1+', r'\1', text)
    text = re.sub(r'\s+', ' ', text).strip().strip('-')

    return text

# ──────────────────────────────────────────────────────────────────────────────
# DATASET
# ──────────────────────────────────────────────────────────────────────────────
class AkkadianTranslationDataset(Dataset):
    def __init__(self, df):
        self.ids = df["id"].tolist()
        self.texts = [
            "translate Akkadian to English: " + t
            for t in df["transliteration"]
        ]

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        return self.ids[idx], self.texts[idx]

def collate_batch(batch):
    ids = [b[0] for b in batch]
    texts = [b[1] for b in batch]

    tokens = tokenizer(
        texts,
        max_length=INFERENCE_CONFIG["maximum_sequence_length"],
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    return ids, tokens

# ──────────────────────────────────────────────────────────────────────────────
# MAIN
# ──────────────────────────────────────────────────────────────────────────────
print("Loading test data...")
test_df = pd.read_csv(INFERENCE_CONFIG["test_data_path"])
test_df["transliteration"] = test_df["transliteration"].apply(preprocess_input_text)
print(f"Loaded {len(test_df)} samples")

print("Loading model & tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    INFERENCE_CONFIG["trained_model_path"],
    use_fast=False      # safer for ByT5 byte-level tokenization
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    INFERENCE_CONFIG["trained_model_path"]
).to(INFERENCE_CONFIG["computation_device"]).eval()

dataset = AkkadianTranslationDataset(test_df)
loader = DataLoader(
    dataset,
    batch_size=INFERENCE_CONFIG["inference_batch_size"],
    shuffle=False,
    num_workers=2,
    collate_fn=collate_batch
)

print("Running inference...")
predictions = []

with torch.inference_mode():
    for ids, inputs in loader:
        outputs = model.generate(
            input_ids=inputs.input_ids.to(INFERENCE_CONFIG["computation_device"]),
            attention_mask=inputs.attention_mask.to(INFERENCE_CONFIG["computation_device"]),
            **INFERENCE_CONFIG["generation_parameters"]
        )

        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        cleaned = [postprocess_translation_output(t) for t in decoded]
        predictions.extend(zip(ids, cleaned))

print("Saving submission...")
submission = pd.DataFrame(predictions, columns=["id", "translation"])
submission.to_csv("submission.csv", index=False)

print("Done ✔")
